In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command = "pwsh -c "../../../../scripts/invoke-dib.ps1 test.dib""
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = None }
00:00:04 #3 [Verbose] >
00:00:04 #4 [Verbose] > ── markdown ────────────────────────────────────────────────────────────────────
00:00:04 #5 [Verbose] > ╭──────────────────────────────────────────────────────────────────────────────╮
00:00:04 #6 [Verbose] > │ # test (Polyglot)                                                            │
00:00:04 #7 [Verbose] > ╰──────────────────────────────────────────────────────────────────────────────╯
00:00:04 #8 [Verbose] >
00:00:04 #9 [Verbose] > ── spiral ──────────────────────────────────────────────────────────────────────
00:00:04 #10 [Verbose] > // // test
00:00:04 #11 [Verbose] >
00:00:04 #12 [Verbose] > open testing
00:00:09 #13 [Verbose] >
00:00:09 #14 [Verbose] > ╭─[ 4.

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(GetExecutableSuffix) test.dib spi } | Invoke-Block

00:00:00 #1 [Debug] writeDibCode / output: Spi / path: test.dib
00:00:00 #2 [Debug] parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:00 #3 [Debug] buildFile / takeWhileInclusive / fsxContent:  / errors: [] / typeErrorCount: 0
00:00:00 #4 [Debug] sendJson / port: 13805 / json: {"FileOpen":{"spiText":"// // # test (Polyglot)\n\nnominal i = ()\nnominal e = ()\nnominal s = ()\nnominal n = ()\nnominal t = ()\nnominal f = ()\nnominal j = ()\nnominal p = ()\n\nunion sensing =\n    | Si : s * i\n    | Se : s * e\n\nunion intuition =\n    | Ni : n * i\n    | Ne : n * e\n\nunion thinking =\n    | Ti : t * i\n    | Te : t * e\n\nunion feeling =\n    | Fi : f * i\n    | Fe : f * e\n\nunion function_stack =\n    | FS : sensing * intuition * thinking * feeling\n\nunion personality_type =\n    | ISTJ : i * s * t * j * function_stack\n    | ISFJ : i * s * f * j * function_stack\n    | INFJ : i * n * f * j * function_stack\n    | INTJ : i * n * t * j * function_stack\n    | IS

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @JacobChang
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 191ms

Started Fable compilation...

Fable compilation finished in 1449ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&m

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

    Finished `release` profile [optimized] target(s) in 24.11s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_temp_test-0f3f76a0f233bf3e.exe)

running 3 tests
test test_parse_number ... ok
2024-03-29T14:04:14.765341Z  INFO spiral_temp_test: input=Comment("$\"|[YMo=%:]$d{/=J(*pD\"$!<")
2024-03-29T14:04:14.765493Z  INFO spiral_temp_test: input=

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
